Following [DSPy: Building an AI Agent with Custom Tools in 40 Lines | Zero to Hero](https://medium.com/@original2547/dspy-building-an-ai-agent-with-custom-tools-in-40-lines-zero-to-hero-4e723580352c)

In [1]:
from dotenv import load_dotenv

import dspy

In [2]:
load_dotenv()

True

In [3]:
lm = dspy.LM("gpt-4.1-nano")
dspy.configure(lm=lm)

In [4]:
def get_weather(city: str) -> str:
    """Get weather data for a city"""
    mock_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 20°C",
        "Chicago": "Rainy, 15°C",
        "Houston": "Thunderstorms, 30°C",
        "Phoenix": "Sunny, 35°C",
    }
    return mock_data.get(city, "Weather data not available")

In [5]:
class AgentTools(dspy.Signature):
    """AI assistant able to use tools"""
    
    input_query: str = dspy.InputField()
    
    history = dspy.InputField(desc="Chat history")
    output = dspy.OutputField(desc="Response to user")

In [6]:
class Agent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.history = []
        self.tools_utils = dspy.ReAct(AgentTools, tools=[get_weather])
    
    def __call__(self, input: str):
        self.add_memory(input)
        str_history = "\n".join(self.history.copy())
        result = self.tools_utils(input_query=input, history=str_history).output
        self.add_memory(result)
        return result
    
    def add_memory(self, memory: str):
        self.history.append(("USER: "if len(self.history) % 2 == 0 else "AI: ") + memory)

In [7]:
chat_module = Agent()

In [8]:
while True:
    human_input = input("You: ")
    print("YOU:\t" + human_input)
    print("AI: \t", chat_module(human_input))

YOU:	Hi what's up?
AI: 	 Hi! Not much, just here to help. How can I assist you today?
YOU:	What can you do?
AI: 	 I am an AI assistant capable of helping you with a wide range of tasks, including answering questions, providing information, assisting with writing or calculations, and more. How can I assist you today?
YOU:	What's the weather forecast for Shin-Osaka?
AI: 	 I'm sorry, but I am currently unable to retrieve the weather forecast for Shin-Osaka. Please try again later or check a weather service for the latest information.
YOU:	And for New York?
AI: 	 The current weather in New York is Sunny with a temperature of 25°C.


KeyboardInterrupt: Interrupted by user